<a href="https://colab.research.google.com/github/egahqusayf/Public-Sentiment-Analysis-of-YouTube-Comments-Toward-Indonesias-Vice-Presidential-Gibran-/blob/main/notebooks/Sentiment_Analysis_Gibran_Rakabuming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment Setup & Dependencies

In [ ]:
!pip install -q torch transformers accelerate datasets sastrawi nltk scikit-learn
!pip install -U transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 46.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6


Import Required Libraries

In [ ]:
# Core Libraries
import re
import numpy as np
import pandas as pd
import torch
import nltk
import matplotlib.pyplot as plt

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support
)

# Indonesian NLP
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


Load Labeled Dataset

In [ ]:
df = pd.read_csv("annotated_comments_labeled.csv")

print("Jumlah data:", len(df))
df.head()


Jumlah data: 2800


,video_id,comment,likes,published_at,label
0,ZhL1ABVBqB8,Bro sori nih..\nSekedar saran aja nih.\nTolong...,0,2025-04-25T18:20:10Z,0.0
1,SzXMacu80o8,1x1 brp woii,0,2025-04-21T12:49:42Z,0.0
2,ZhL1ABVBqB8,Senantiasa Sehat Panjang Umur Mas Wapres 💙🤲,1,2025-04-25T13:57:38Z,1.0
3,SzXMacu80o8,"Katanya kan mewakili anak muda, tapi kebanyaka...",0,2025-04-21T10:41:31Z,0.0
4,kJks8qSH3ko,"Semangat pak Wapres, FAKTA tak terbantahkan kl...",7,2025-04-25T07:58:07Z,-1.0


Text Cleaning & Preprocessing

In [ ]:
nltk.download("punkt")
stopwords = set(StopWordRemoverFactory().get_stop_words())

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_comment"] = df["comment"].astype(str).apply(clean_text)
df = df[df["clean_comment"] != ""]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Label Encoding

In [ ]:
label_map = {
    -1: 0,
     0: 1,
     1: 2
}

df["label_id"] = df["label"].map(label_map)

Train–Validation Split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["clean_comment"],
    df["label_id"],
    test_size=0.2,
    stratify=df["label_id"],
    random_state=42
)


Tokenization

In [ ]:
MODEL_NAME = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: xlm-roberta-base
Key                         | Status     | 
----------------------------+------------+-
lm_head.layer_norm.bias     | UNEXPECTED | 
roberta.pooler.dense.bias   | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
roberta.pooler.dense.weight | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
lm_head.bias                | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
classifier.out_proj.weight  | MISSING    | 
classifier.out_proj.bias    | MISSING    | 
classifier.dense.bias       | MISSING    | 
classifier.dense.weight     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
train_encodings = tokenizer(
    train_texts.tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    val_texts.tolist(),
    truncation=True,
    padding=True,
    max_length=128
)


Custom Dataset Class

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

Evaluation Metrics Function

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="weighted"
    )
    acc = accuracy_score(labels, predictions)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


Training Configuration

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=100
)


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Model Training & Evaluation

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
eval_result = trainer.evaluate()

print("Evaluation Result:")
for k, v in eval_result.items():
    print(f"{k}: {v:.4f}")

Validation Prediction & Performance Report

In [ ]:
predictions = trainer.predict(val_dataset)

logits = predictions.predictions
y_true = predictions.label_ids
y_pred = np.argmax(logits, axis=1)

label_names = ["Negative", "Neutral", "Positive"]

print(classification_report(
    y_true,
    y_pred,
    target_names=label_names,
    digits=4
))


Confusion Matrix Visualization

In [ ]:
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=label_names,
    yticklabels=label_names
)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


Save Trained Model

In [ ]:
SAVE_PATH = "./best_model1"

trainer.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print(f"✅ Model dan tokenizer disimpan di: {SAVE_PATH}")


Single Text Inference Function

In [ ]:
import torch
from torch.nn.functional import softmax

LABEL_MAP = {
    0: "Negatif",
    1: "Netral",
    2: "Positif"
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)

    confidence, prediction = torch.max(probs, dim=1)

    return {
        "label_id": prediction.item(),
        "label": LABEL_MAP[prediction.item()],
        "confidence": confidence.item()
    }


In [ ]:
sample_text = "Saya tidak memihak siapapun, tapi memang pemerintahan kali ini perlu dievaluasi besar-besaran"

result = predict_sentiment(sample_text)

print(f"Text       : {sample_text}")
print(f"Sentiment  : {result['label']}")
print(f"Confidence : {result['confidence']:.4f}")


Load Unlabeled YouTube Comments

In [ ]:
UNLABELED_PATH = "youtube_comments_unlabeled.csv"
df_unlabeled = pd.read_csv(UNLABELED_PATH)

print("Kolom dataset:", df_unlabeled.columns.tolist())
df_unlabeled.head()


Preprocess & Sample Unlabeled Data

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df_sample = df_unlabeled.reset_index(drop=True)

df_sample["clean_comment"] = df_sample["comment"].apply(clean_text)

texts = df_sample["clean_comment"].tolist()

print("Jumlah komentar untuk inference:", len(texts))

Batch Inference on Unlabeled Data

In [ ]:
from tqdm import tqdm

label_map = {0: "negative", 1: "neutral", 2: "positive"}

pred_labels = []
confidences = []

model.eval()

for text in tqdm(texts, desc="Running inference"):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)

    conf, pred = torch.max(probs, dim=1)
    pred_labels.append(label_map[pred.item()])
    confidences.append(conf.item())

df_sample["predicted_sentiment"] = pred_labels
df_sample["confidence"] = confidences


Sentiment Distribution Visualization

In [ ]:
sentiment_counts = df_sample["predicted_sentiment"].value_counts()
print(sentiment_counts)

In [ ]:
plt.figure(figsize=(6, 4))
sentiment_counts.plot(kind="bar")
plt.title("Sentiment Distribution of All YouTube Comments")
plt.xlabel("Sentiment")
plt.ylabel("Number of Comments")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# ===== SETTING DISPLAY AGAR TEKS TIDAK TERPOTONG =====
pd.set_option("display.max_colwidth", None)

# ===== PARAMETER FILTER =====
CONF_THRESHOLD = 0.90
MIN_WORDS = 20
SENTIMENTS = ["negative", "neutral", "positive"]

# ===== TAMPILKAN KOMENTAR =====
for sent in SENTIMENTS:
    print(f"\n===== {sent.upper()} (confidence > {CONF_THRESHOLD}, ≥ {MIN_WORDS} kata) =====\n")

    filtered_df = df_sample[
        (df_sample["predicted_sentiment"] == sent) &
        (df_sample["confidence"] > CONF_THRESHOLD) &
        (df_sample["comment"].str.split().str.len() >= MIN_WORDS)
    ][["comment", "confidence"]]

    display(filtered_df.head(15))
